In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.manifold import TSNE
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

###### 1. Make python doc2vec model

In [2]:
#파이선 doc2vec 모델 학습, 128차원 고정 길이의 벡터로 만듦, epochs 40 -> 300

def make_doc2vec_models(tagged_data, vector_size = 128, window = 3, epochs = 300, min_counts = 0, workers = 4):
  model = Doc2Vec(tagged_data, vector_size = vector_size, window = window, epochs = epochs, min_counts = min_counts, workers = workers)
  return model

###### 2. Make doc2vec data

In [3]:
# Doc2Vec에서 사용할 수 있는 데이터 생성
# Doc2Vec model 학습에 사용됩니다.

def make_doc2vec_webtoon(data, column, tagged_document = False):
  data_doc = []
  for tag, doc in zip(data.index, data[column]):
    doc = doc.split(" ")
    data_doc.append(([tag], doc))
  if tagged_document:
    data = [TaggedDocument(words = text, tags = tag) for tag, text in data_doc]
    return data
  else:
    return data_doc

###### 3. User embedding

In [4]:
# User embedding 생성
# User history를 평균해서 user embedding을 만드는 함수

def make_user_embedding(index_list, data_doc, model):
    user = []
    user_embedding = []
    for i in index_list:
        user.append(data_doc[i][0][0])
    for i in user:
        user_embedding.append(model.docvecs[i])
    user_embedding = np.array(user_embedding)
    user = np.mean(user_embedding, axis = 0)
    return user

###### 4. Recommendation Top 5 

In [5]:
# 추천 결과 반환, 상위 5개를 추천해주는 함수

def recommendation_webtoon(user, data_doc, model):
  scores = []

  for tags, text in data_doc:
    train_doc_vec = model.docvecs[tags[0]]
    scores.append(cosine_similarity(user.reshape(-1,128), train_doc_vec.reshape(-1,128)))

  scores = np.array(scores).reshape(-1)
  scores = np.argsort(-scores)[:5]

  return webtoon.loc[scores, :]

###### 5. Check contents

In [6]:
#랜덤하게 뽑아낸 사용자가 본 컨텐츠

def user_contents(data):
  print(data[['title']])

In [7]:
from google.colab import files
myfile = files.upload()

Saving Webtoon_cleaned.csv to Webtoon_cleaned.csv


###### 7. See csv file

In [8]:
webtoon = pd.read_csv('Webtoon_cleaned.csv')
webtoon.head()


,Unnamed: 0,title,genre,story,title_genre_story,komoran
0,0,참교육,액션,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다부활남 채용택 ...,참교육 액션 무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다부...,"['참', '교육', '액션', '교권', '교권', '보호국', '소속', '진의..."
1,1,신의 탑,판타지,자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년그리고 그런 소년을 시험하는 탑,신의 탑 판타지 자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년그리고 그런 소년...,"['신의 탑', '판타지', '자신', '것', '소녀', '탑', '소년', '소..."
2,2,뷰티풀 군바리,드라마,여자도 군대에 간다면본격 여자도 군대 가는 만화,뷰티풀 군바리 드라마 여자도 군대에 간다면본격 여자도 군대 가는 만화,"['뷰티풀', '군바리', '드라마', '여자도', '군대', '간다', '면', ..."
3,3,소녀의 세계,드라마,완벽해 보이지만 사실 외로웠던 백조들과 맘씨 착한 오리가 만나여러 갈등을 함께 겪으...,소녀의 세계 드라마 완벽해 보이지만 사실 외로웠던 백조들과 맘씨 착한 오리가 만나여...,"['소녀', '세계', '드라마', '완벽', '사실', '백조', '맘', '씨'..."
4,4,팔이피플,드라마,마스크걸 위대한 방옥숙 매미희세 작가의 신작에서 육아용품 파는 평범한 유부녀 박주연...,팔이피플 드라마 마스크걸 위대한 방옥숙 매미희세 작가의 신작에서 육아용품 파는 평범...,"['팔이', '피플', '드라마', '마스크', '거', '방', '옥', '숙',..."


###### 8. title_genre_story vs Komoran

In [9]:
webtoon_comb_tag = make_doc2vec_webtoon(webtoon,'title_genre_story', tagged_document=True)
#print(webtoon_comb_tag)
webtoon_comb = make_doc2vec_webtoon(webtoon, 'title_genre_story')
#print(webtoon_comb)
webtoon_nouns_tag = make_doc2vec_webtoon(webtoon, 'komoran', tagged_document=True)
#print(webtoon_nouns_tag)
webtoon_nouns = make_doc2vec_webtoon(webtoon, 'komoran')
#print(webtoon_nouns)
#tag가 붙은 데이터는 doc2vec model 학습에 사용되며, 없는 데이터는 user embedding, cosine similarity를 구할 때 사용됩니다.

###### 9. Using function [Make python doc2vec model]

In [10]:
# Doc2Vec model 만들기
model_comb = make_doc2vec_models(webtoon_comb_tag)
model_nouns = make_doc2vec_models(webtoon_nouns_tag)

###### 9-1. Pick random contents

In [11]:
user1 = webtoon.loc[random.sample(webtoon.loc[webtoon.genre == '액션', :].index.values.tolist(),5),:]
user_contents(user1)
#user1,2,3 각 장르 user contents 를 만드는 과정

         title
64   보살님이 캐리해!
332         광장
238     야만의 시대
428       생존로그
334     초인의 시대


In [12]:
user2 = webtoon.loc[random.sample(webtoon.loc[webtoon.genre == '스릴러', :].index.values.tolist(),5),:]
user_contents(user2)

         title
449       가짜인간
75        엽총소년
204  묵시의 인플루언서
7         파이게임
377       더 나우


In [13]:
user3 = webtoon.loc[random.sample(webtoon.loc[webtoon.genre == '판타지', :].index.values.tolist(),5),:]
user_contents(user3)

          title
233        길티액스
209  나 혼자 네크로맨서
407    살아남은 로맨스
375       도사 가온
314         트럼프


### Result : title_genre_story

In [14]:
user_action = make_user_embedding(user1.index.values.tolist(), webtoon_comb,model_comb)
user_thriller = make_user_embedding(user2.index.values.tolist(), webtoon_comb, model_comb)
user_fantasy = make_user_embedding(user3.index.values.tolist(), webtoon_comb, model_comb)
#각 장르마다 embedding을 통해서 128개의 평균을 구하게 됩니다.

In [15]:
result = recommendation_webtoon(user_action, webtoon_comb, model_comb)
pd.DataFrame(result.loc[:, ['title','genre', 'story']])

,title,genre,story
219,최강전설 강해효,액션,맹수같이 거친 문제아 학생들만 모여있는 최강고등학교그곳에서 변화가 시작되었다
6,장씨세가 호위무사,액션,당신이 부른 것이오 나란 사람을 은둔고수 광휘 호위무사 되다웹소설 원작 웰메이드 무...
334,초인의 시대,액션,지구에 근원 모를 괴물들이 출현하고그들을 제압하는 초인들 또한 등장한다정체불명의 괴...
151,고삼무쌍,액션,천년간 사라진 무림이 현대 고등학교를 배경으로 되살아난다무림인들의 고교습격 장르파...
135,헬퍼 2 : 킬베로스,액션,역시는 역시 역시군킬베로스 트라이브의 대장 장광남이 펼치는 스트릿 액션 만화


In [16]:
result = recommendation_webtoon(user_thriller, webtoon_comb, model_comb)
pd.DataFrame(result.loc[:, ['title','genre', 'story']])

,title,genre,story
7,파이게임,스릴러,시간이 흐르면 돈이 쌓인다피가 흐르면 시간이 쌓인다과연 최후에 남을 상금과 승자는 ...
169,아도나이,스릴러,외계인을 믿는 사이비종교 심층 취재기이 종교 알면 알수록 위험하다
302,악몽일기,스릴러,섬뜩한 악몽을 기록해둔 옴니버스식 공포 스릴러과연 이것은 악몽일까 현실일까평범했던 ...
449,가짜인간,스릴러,폭력을 일삼던 엄마의 손에서 벗어나 동생 수아와 단둘이 살게 된 지아는무슨 일이 있...
29,꿈의 기업,스릴러,문명을 지배하는 거대기업과 거대기업을 움직이는 인공지능그 인공지능이 꿈을 꾸기 시작했다


In [17]:
result = recommendation_webtoon(user_fantasy, webtoon_comb, model_comb)
pd.DataFrame(result.loc[:, ['title','genre', 'story']])

,title,genre,story
233,길티액스,판타지,눈앞의 불의를 외면하는 순간 반드시 불행한 사람이 생긴다정의로운 소년 단태는 친구를...
431,잉여특공대,판타지,뽀짝이 특공대원들의 슈퍼히어로맨스
375,도사 가온,판타지,도사가 되고픈 소년 가온과 무명도사의 도술액션 모험판타지
257,카루나,판타지,지옥의 군대와 맞서는 항마승병 카루나카루나에 도전하는 소년액션 모험 판타지
390,먹지마세요,판타지,달콤한 복숭아 소년요괴와 귀신들에게 쫓기다


#### Result : Komoran

In [18]:
user_action = make_user_embedding(user1.index.values.tolist(), webtoon_nouns,model_nouns)
user_thriller = make_user_embedding(user2.index.values.tolist(), webtoon_nouns, model_nouns)
user_fantasy = make_user_embedding(user3.index.values.tolist(), webtoon_nouns, model_nouns)

In [19]:
result = recommendation_webtoon(user_action, webtoon_nouns, model_nouns)
pd.DataFrame(result.loc[:, ['title','genre', 'story']])

,title,genre,story
334,초인의 시대,액션,지구에 근원 모를 괴물들이 출현하고그들을 제압하는 초인들 또한 등장한다정체불명의 괴...
332,광장,액션,군계일학 정점에서 조직을 떠난 기준년만에홀로 복귀한다
238,야만의 시대,액션,에서 나와 똑같이 생긴 사람을 발견했다 과거도 미래도 없이 막 나가던 내 인생과 달...
71,한림체육관,액션,세상에서 가장 강한 남자가 되고 싶다그릇된 강함을 실천하던 전영하가 한림체육관에 들...
64,보살님이 캐리해!,액션,억겁의 시간을 넘어 환생한 보살들이여 야차들의 습격에서 평화로운 일상과 법화를 수호하라


In [20]:
result = recommendation_webtoon(user_thriller, webtoon_nouns, model_nouns)
pd.DataFrame(result.loc[:, ['title','genre', 'story']])

,title,genre,story
449,가짜인간,스릴러,폭력을 일삼던 엄마의 손에서 벗어나 동생 수아와 단둘이 살게 된 지아는무슨 일이 있...
169,아도나이,스릴러,외계인을 믿는 사이비종교 심층 취재기이 종교 알면 알수록 위험하다
75,엽총소년,스릴러,약자에서 영웅이 될 수밖에 없었던 소년 한규환 그리고 그 소년을 영웅으로 만들어줄 ...
377,더 나우,스릴러,년 전의 순진한 나를 만났다내가 다 고쳐줄게 우리 인생과거 사건으로 인한 트라우마로...
234,로그아웃,스릴러,인간이 상상하기 힘든 신기술들이 개발되고욕망을 충족해가는 사람들 인류에게 주어진 선...


In [21]:
result = recommendation_webtoon(user_fantasy, webtoon_nouns, model_nouns)
pd.DataFrame(result.loc[:, ['title','genre', 'story']])

,title,genre,story
343,어글리후드,판타지,외계인이 유일신으로 군림한 미친 세상 바꾸고싶다면 사탄이 되어라
375,도사 가온,판타지,도사가 되고픈 소년 가온과 무명도사의 도술액션 모험판타지
78,신도림,액션,세상이 무너지고 세워진 지하도시 신도림그곳에 들어가지 못하고 생존한 소년들이 지하도...
314,트럼프,판타지,사형 위기에 직면한 수배범 아버지와 세상 누구보다 존경하는 단장님 사이소년의 갈등이...
431,잉여특공대,판타지,뽀짝이 특공대원들의 슈퍼히어로맨스
